# LightGBMの推論用カスタムコンテナを構築し、SageMakerによる推論の仕組みを深く理解する

このノートブックでは、LightGBMがインストールされたカスタムコンテナ構築し、SageMaker Trainingジョブで学習後、推論を行います。
カスタムコンテナの挙動を観察し、SageMakerの推論動作について理解を深めます。

ノートブックは20分程度で実行できます。

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [ ]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

In [ ]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

In [ ]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.113.0

# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [ ]:
import sklearn
sklearn.__version__ # 1.0.1

In [ ]:
import pandas as pd
pd.__version__ # 1.3.4

## 1-1. データロード

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [ ]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [ ]:
# 確認
print(trainX.shape)
trainX.head()

In [ ]:
# 確認
print(valX.shape)
valX.head()

In [ ]:
# 確認
print(testX.shape)
testX.head()

In [ ]:
# 確認
y_test[0:5]

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [ ]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [ ]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

Session().default_bucket()を実行することで、

sagemaker-\<リージョン名>-\<アカウントID> という名称のバケットが作成されます。

https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/session.py



In [ ]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [ ]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

In [ ]:
# 確認:格納したS3のURIが返されています
print(train_s3)
print(valid_s3)

# 2.LightGBMカスタムコンテナの構築
推論用のカスタムコンテナの作成には大きく分けて4つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-inference/


\<参考>学習用カスタムコンテナの作成

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

今回は、学習編で作成した、学習（学習編blogのパターン3）+推論（パターン4）用のカスタムコンテナを利用します。

ベースイメージ(ubuntu:16.04) の他は、カスタマイズします。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

まずは、Dockerfileを確認します。

In [ ]:
!pygmentize ./container/Dockerfile

### 推論エンドポイント構築時のSageMakerの動作について
SageMakerの推論エンドポイントのデプロイは、SageMaker SDKでは、deploy()メソッドで実行します。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

その際に、SageMakerは以下のコマンドを実行します。

docker run \<Docker image> server

今回のカスタムコンテナでは、 /opt/program に配置した serve スクリプトが実行されます。

serveスクリプトを確認してみましょう。

In [ ]:
!pygmentize -l py ./container/lightgbm_regression/serve

初めに、末尾の start_server() が実行され、start_server()では以下が行われます。

* nginxの起動（Webサーバ/リバースプロキシの役割）
    * nginx.confを読み込みます。
* gunicornの起動（Applicationサーバの役割）
    * gunicornの起動コマンド引数に'wsgi:app'とあるように、wsgiモジュールwsgi.pyの、appアプリケーションを読み込みます。

nginx.confを確認してみましょう。

In [ ]:
!pygmentize ./container/lightgbm_regression/nginx.conf

SageMakerから受け取った /ping と /invocations リクエストを上記で設定したgunicornに渡します。
以下に記載があるように、ポート8080を利用する必要があります。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

How Containers Serve Requests  
Containers need to implement a web server that responds to /invocations and /ping on port 8080.

次に、gunicornへのアプリケーションのキック用に使われるファイル wsgi.pyを確認します。

predictor.py の、appを読み込んでいることがわかります。

In [ ]:
!pygmentize ./container/lightgbm_regression/wsgi.py

predictor.py を確認します。

flaskフレームワークを用いて、/ping、 /invocations に対する処理を実装していることがわかります。

In [ ]:
!pygmentize ./container/lightgbm_regression/predictor.py

## 2-2. dockerイメージの build & push
上記で確認したカスタムコンテナをビルドします。

build & pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを設定します。

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
# imageURLの設定
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [ ]:
# 確認
image_uri

In [ ]:
# 学習で指定するLightGBMのハイパーパラメータを設定します。
hyperparameters={'boosting_type': 'gbdt',
                 'objective': 'regression',
                 'num_leaves': 31,
                 'learning_rate': 0.05,
                 'feature_fraction': 0.9,
                 'bagging_fraction': 0.8,
                 'bagging_freq': 5,
                 'verbose': 0
                }

## 2-4.ローカル学習の実行
まずはローカルモードでモデルの学習を行います。
ローカルモードを利用することで、コンテナイメージのダウンロードや展開の手間を省くことができる、デバッグを行う場合に便利です。

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [ ]:
from sagemaker.estimator import Estimator

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
# 確認
role

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [ ]:
local_lightgbm = Estimator(image_uri,
                           role,
                           instance_count=1,
                           instance_type="local",
                           hyperparameters=hyperparameters
                          )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果についてもS3に保管されます。

s3://sagemaker-< リージョン名 >-< アカウントID >/sagemaker-lightgbm-regression-yyyy-MM-dd-HH-mm-ss-fff/

* model.tar.gz
* output.tar.gz

SageMaker Trainingジョブの詳細については、BlackBeltの解説もご参照ください。
https://www.youtube.com/watch?v=byEawTm4O4E

## 2-5.ローカルデプロイ

インプットデータの形式を指定するSerializerは、CSVSerializerとします。
SerializerとDeserializerについて学ぶ場合は、study_serializer_deserializer.ipynb を参考にしてください。

https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
# 事前準備：全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
# 確認
!docker ps

起動中のコンテナイメージがないことを確認し、ローカルデプロイを行います。

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
# 確認
!docker ps

ローカルにコンテナイメージが展開されていることが確認できました。

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(type(predicted))

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

Estimatorの引数instance_typeにインスタンスタイプを指定することで、学習ジョブが発行されます。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(image_uri,
                         role,
                         instance_count=1,
                         instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
                         hyperparameters=hyperparameters
                        )

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

Billable seconds: \<秒数>
が課金対象の時間となります。

## 2-8.エンドポイントにデプロイ

学習ジョブで構築されたモデルをデプロイします。
デプロイ時にSageMaker は 

docker run \<image> serve

を実行します。

    
デプロイには3分ほどかかります。

In [ ]:
#from sagemaker.predictor import csv_serializer # SageMaker SDK v1以前の使い方
# https://sagemaker.readthedocs.io/en/stable/v2.html

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
#predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True) # SageMaker SDK v1以前の使い方
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=sagemaker.serializers.CSVSerializer(), wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3.推論コードを外部ファイルとして指定できるようにする
推論コードを外部から指定するために、SageMaker Inference Toolkitを導入します。以下のブログのパターン2に該当します。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-inference/

https://github.com/aws/sagemaker-inference-toolkit

SageMaker Inference Toolkitは、MMS(Multi Model Server)の利用を前提としているため、MMSも導入します。

前セクションではnginx, gunicorn, flaskで推論仕組みを構築しましたが、モデルサービングの仕組みはMMSを利用します。

https://github.com/awslabs/multi-model-server/tree/master/docker

* SageMaker-Inference-Toolkitと、Multi Model Serverを導入する
* ビルトインコンテナ + requirements.txt, inference.pyを利用する

MMSの利用については、以下のサンプルコードも参照ください。

https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own


## 3-1.Dockerfileの確認

まずは、利用するDockerfileを確認します。
MMSに必要なJavaをインストールし、MMSとinference-toolkitをインストールしています。

LightGBMはrequirements.txtでインストールを試みるため、Dockerfileには記載していません。（前セクションのようにDockerfileに記載することも可能）

In [ ]:
!pygmentize ./container_sminftoolkit/Dockerfile

## 3-2.エントリポイントを確認

SageMakerSDKにてdeploy()を実行した際の

docker run \<image> server

で実行される、ENTRYPOINTを確認します。

dockerd-entrypoint.py が実行されることがわかります。このファイルを確認してみましょう。

In [ ]:
!pygmentize ./container_sminftoolkit/dockerd-entrypoint.py

これは、以下に該当する model_server.py の、start_model_server()が実行されます。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

その後の処理の流れはノートブックの末尾に記載します。

では、このDockerfileをbuildします。

build&pushには3分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
#algorithm_name=demo-sagemaker-multimodel
algorithm_name=demo-sagemaker-inftoolkit

#cd container
cd container_sminftoolkit

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 3-3.ローカルにエンドポイントをデプロイ
モデルは前のセクションで作成したLightGBMのモデルを利用します。

* 推論ロジックを記載したファイル inference.py を指定する
* LightGBMライブラリはrequirements.txtでインストールする

推論ロジックを記載した inference.py を確認します。

* model_fn : deploy()実行時にモデルロードのため利用されます
* input_fn : 推論時に実行され、前処理を行います。
* predict_fn : 推論時に実行され、推論を行います。
* output_fn : 推論時に実行され、後処理を行います。

model_fnにおいて、LightGBMのモデルが.txt形式なので、lgb.Booster(model_file='.txt')でモデルをロードします。

出力ログから動作フローを追えるように、参考情報を多く出力しています。

In [ ]:
!pygmentize ./src_builtin_container_serve/inference.py

次に requirements.txt を確認します。インストールする lightgbm のみ記載しています。

In [ ]:
!pygmentize ./src_builtin_container_serve/requirements.txt

In [ ]:
container_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/demo-sagemaker-inftoolkit:latest'

In [ ]:
container_uri

In [ ]:
### 2.8の学習ジョブで構築したモデルを利用する
#est_lightgbm.model_data

### ローカル学習で構築したモデルを利用する場合
model_data=local_lightgbm.model_data

In [ ]:
!docker ps

In [ ]:
#全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
!docker ps

predictor_clsには、SageMaker SDK v2のPredictorクラスを利用

SageMaker SDK v1.72.0
https://sagemaker.readthedocs.io/en/v1.72.0/api/inference/predictors.html

SageMaker SDK v2.115
https://sagemaker.readthedocs.io/en/v2.115.0/api/inference/predictors.html

In [ ]:
from sagemaker.predictor import Predictor

lgb_model = sagemaker.model.Model(#est_xgb.image_uri, # XGBoostビルトインコンテナのURI
                                  container_uri,
                                  model_data=model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=Predictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                  #entry_point='./src_builtin_container_serve/inference.py'
                                 )

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=sagemaker.serializers.CSVSerializer(), ### string形式でSageMakerに渡す（認識してもらう）
                                      )

In [ ]:
# 参考：SageMaker SDK v1以前の場合
#from sagemaker.predictor import RealTimePredictor
#
#lgb_model = sagemaker.model.Model(#est_xgb.image_uri, # XGBoostビルトインコンテナのURI
#                                  container_uri,
#                                  model_data=model_data, # ローカル学習で生成したモデルファイル
#                                  role=role,
#                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
#                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
#                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
#                                  #entry_point='./src_builtin_container_serve/inference.py'
#                                 )

In [ ]:
#predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
#                                       instance_type='local', 
#                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
#                                      )

In [ ]:
!docker ps

## 3-4.推論実施

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(type(predicted))
print('='*50)
print(predicted)
print('='*50)
predicted

# (optional) 4.XGBoostコンテナで、LightGBMの推論を実施する

XGBoostのビルトインコンテナでLightGBMの推論を行います。
LightGBMはrequirements.txtでインストールします。

カスタムコンテナではなく、ビルトインコンテナの利用なので、ブログのパターンには該当しません。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-inference/


\<参考>LightGBMはSageMakerのビルトインアルゴリズムとして用意されています。

https://docs.aws.amazon.com/sagemaker/latest/dg/lightgbm.html    

## 4-1. 既存のモデルを利用

In [ ]:
xgb_container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [ ]:
xgb_container_uri

In [ ]:
lgb_model = sagemaker.model.Model(xgb_container_uri, # XGBoostビルトインコンテナのURI
                                  model_data=model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=Predictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                 )

## 4-2.ローカルにデプロイ

In [ ]:
!docker ps

In [ ]:
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=sagemaker.serializers.CSVSerializer(), ### string形式でSageMakerに渡す（認識してもらう）
                                       #deserializer=None, 
                                      )

## 4-3.推論実施

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
# 確認
print(type(predicted))
print('='*50)
print(predicted)
print('='*50)
predicted

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# Tips
* 外部指定する推論用ファイルは、.py である必要がある。また、関数名も決まっている。
    * MMSは.pyを扱うように設計されているため。
    * https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit
* Amazon SageMaker におけるカスタムコンテナ実装パターン詳説 〜学習編〜
    * https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/
* Amazon SageMaker におけるカスタムコンテナ実装パターン詳説 〜推論編〜
    * https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-inference/
* Building your own algorithm container
    * nginx  + gunicorn + flask で構成する推論コンテナについて説明しています
    * https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb


# 解説：start_model_server()後の挙動について
3-2. で実行した、start_model_server()のその後の挙動について詳細を解説します。

In [ ]:
# 3-2.で確認したファイル
!pygmentize ./container_sminftoolkit/dockerd-entrypoint.py

これは、以下に該当する model_server.py の、start_model_server()が実行されます。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

start_model_server()は、引数を指定しない場合、SageMaker Inference Toolkitのハンドラサービスを利用します。

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py

    The handler service is responsible for defining an ``initialize`` and ``handle`` method.
        - The ``handle`` method is invoked for all incoming inference requests to the model server.
        - The ``initialize`` method is invoked at model server start up.

データが入力されたときにhandle()が実行される。  
モデルサーバーが起動した時に、initialize()が実行される。

initialize()では、Transform()のvalidate_and_initialize()が実行され、ユーザー定義の関数が読み込まれる。

inference-toolkitのTransformer()は以下

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/transformer.py

validate_and_initialize() -> _validate_user_module_and_set_functions() にて、ユーザー定義の関数が読み込まれる。  
validate_and_initialize()内で、モデルのロードも実行される。



## DEFAULT_HANDLER_SERVICE がmulti-model-serverコマンドに読み込まれるまで
start_model_server(handler_service=DEFAULT_HANDLER_SERVICE)　とありますが、handler_serviceはサーバー起動のコマンドmulti-model-serverにどのように渡されているのでしょうか？
追ってみましょう

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

_create_model_server_config_file(env, handler_service_for_config)

が実行されています。これは、configuration_properties にhander_serverなどを追記して、

utils.write_file(MMS_CONFIG_FILE, configuration_properties)で、MMS_CONFIG_FILEに書き込みます。
このMMS_CONFIG_FILEが、muti-model-serverコマンド実行時に引数として指定されます。

## ハンドラサービスと推論ハンドラについて
ハンドラサービスと推論ハンドラがある。

https://github.com/aws/sagemaker-inference-toolkit

ハンドラサービスは、SageMaker Inference Toolkit のGitHubにある以下に該当する。

2.Implement a handler service that is executed by the model server.

推論ハンドラは、以下に該当する。

1.Implement an inference handler, which is responsible for loading the model and providing input, predict, and output functions. 

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_inference_handler.py

default_model_fn()などが定義されている。

2.のハンドラサービスから、1.の推論ハンドラがロードされる。推論ハンドラはinference-toolkitで用意したものを使ってもよい。